In [1]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\vedant girawale\appdata\local\programs\python\python312\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [2]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\vedant girawale\appdata\local\programs\python\python312\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [3]:
##important one so far

import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose Estimation
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Define key landmarks for yoga alignment (e.g., Warrior II)
golden_keypoints = np.array([
    [0.45, 0.3],  # left_shoulder
    [0.55, 0.3],  # right_shoulder
    [0.4, 0.5],   # left_hip
    [0.6, 0.5],   # right_hip
    [0.35, 0.7],  # left_knee
    [0.65, 0.7],  # right_knee
    [0.3, 0.9],   # left_ankle
    [0.7, 0.9],   # right_ankle
])

# Scale golden keypoints to match frame dimensions
def scale_golden_keypoints(frame_width, frame_height):
    scaled_keypoints = golden_keypoints.copy()
    scaled_keypoints[:, 0] *= frame_width
    scaled_keypoints[:, 1] *= frame_height
    return scaled_keypoints

# Function to calculate Euclidean distance for pose accuracy
def calculate_pose_accuracy(user_keypoints, golden_keypoints):
    distances = np.linalg.norm(user_keypoints - golden_keypoints, axis=1)
    return distances.mean()

# Function to provide alignment feedback
def provide_alignment_feedback(user_keypoints, golden_keypoints, threshold=20):
    feedback = []
    keypoints = ['left_shoulder', 'right_shoulder', 'left_hip', 'right_hip',
                 'left_knee', 'right_knee', 'left_ankle', 'right_ankle']
    
    for i, (user_kp, golden_kp) in enumerate(zip(user_keypoints, golden_keypoints)):
        diff_x, diff_y = user_kp - golden_kp
        direction = []
        if abs(diff_x) > threshold:
            direction.append("move left" if diff_x > 0 else "move right")
        if abs(diff_y) > threshold:
            direction.append("move up" if diff_y > 0 else "move down")
        if direction:
            feedback.append(f"{keypoints[i]}: {' and '.join(direction)}")
    
    return feedback if feedback else ["Great Job!"]

# Load the reference video
reference_video_path = 'WhatsApp Video 2024-10-26 at 19.38.12_ccc7f76d.mp4'  # Update with actual video path
reference_cap = cv2.VideoCapture(reference_video_path)

# Get native resolution for the reference video
ref_width = int(reference_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
ref_height = int(reference_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Initialize camera feed
user_cap = cv2.VideoCapture(0)

if not user_cap.isOpened():
    print("Error: User camera feed could not be opened.")
if not reference_cap.isOpened():
    print("Error: Reference video could not be loaded.")

alignment_threshold = 190  # Pose accuracy threshold
total_accuracy = 0
frame_count = 0
pause_ref_video = False
last_ref_frame = None

while reference_cap.isOpened() and user_cap.isOpened():
    # Capture the reference video frame only if pose is correct
    if not pause_ref_video:
        ref_ret, ref_frame = reference_cap.read()
        if not ref_ret:
            print("Reference video completed.")
            break  # End video playback after one run
        last_ref_frame = ref_frame.copy()  # Save last frame for pausing
    else:
        # Keep displaying the last frame if reference video is paused
        ref_frame = last_ref_frame

    # Capture the user camera frame
    user_ret, user_frame = user_cap.read()
    if not user_ret:
        print("Error reading from user camera. Exiting.")
        break
    
    # Flip the user frame for a selfie view
    user_frame = cv2.flip(user_frame, 1)
    h, w, _ = user_frame.shape  # Full-sized user frame

    # Convert user frame to RGB and process for pose landmarks
    rgb_frame = cv2.cvtColor(user_frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)
    
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(user_frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        # Extract keypoints for yoga alignment
        user_keypoints = np.array([
            [results.pose_landmarks.landmark[i].x * w, results.pose_landmarks.landmark[i].y * h]
            for i in [11, 12, 23, 24, 25, 26, 27, 28]  # Indices for shoulders, hips, knees, ankles
        ])
        
        # Calculate pose accuracy and provide feedback
        accuracy = calculate_pose_accuracy(user_keypoints, scale_golden_keypoints(w, h))
        total_accuracy += accuracy
        frame_count += 1
        feedback_text = provide_alignment_feedback(user_keypoints, scale_golden_keypoints(w, h))
        
        # Control video play/pause based on pose accuracy
        if accuracy < alignment_threshold:
            feedback_display_text = "Great Job!"
            color = (0, 255, 0)
            pause_ref_video = False  # Continue advancing the reference video
        else:
            color = (0, 0, 255)
            pause_ref_video = True  # Pause the reference video

        # Display feedback text on user frame, line-by-line
        y_offset = 50
        for line in feedback_text:
            cv2.putText(user_frame, line, (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 1)
            y_offset += 25  # Space out each feedback line

        # Display pose accuracy
        cv2.putText(user_frame, f"Pose Accuracy: {accuracy:.2f}", (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

    # Resize reference video overlay for portrait mode and move it to top right
    overlay_width = int(w * 0.3)  # Smaller for portrait layout
    overlay_height = int(overlay_width * ref_height / ref_width)  # Maintain aspect ratio
    ref_frame_resized = cv2.resize(ref_frame, (overlay_width, overlay_height))
    user_frame[10:10 + overlay_height, w - overlay_width - 10:w - 10] = ref_frame_resized

    # Display the user frame in a fullscreen window optimized for portrait mode
    cv2.namedWindow("Yoga Alignment Assistant", cv2.WINDOW_NORMAL)
    cv2.setWindowProperty("Yoga Alignment Assistant", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    cv2.imshow("Yoga Alignment Assistant", user_frame)
    
    # Quit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Calculate the final score
if frame_count > 0:
    average_accuracy = total_accuracy / frame_count
    score = max(0, 100 - average_accuracy)  # Scaled score out of 100
    print(f"Exercise Score: {score:.2f}%")
else:
    print("No frames were processed.")

# Release resources
user_cap.release()
reference_cap.release()
cv2.destroyAllWindows()
pose.close()


MemoryError: 

In [24]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose Estimation
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Define key landmarks for yoga alignment (e.g., Warrior II)
golden_keypoints = np.array([
    [0.45, 0.3],  # left_shoulder
    [0.55, 0.3],  # right_shoulder
    [0.4, 0.5],   # left_hip
    [0.6, 0.5],   # right_hip
    [0.35, 0.7],  # left_knee
    [0.65, 0.7],  # right_knee
    [0.3, 0.9],   # left_ankle
    [0.7, 0.9],   # right_ankle
])

# Scale golden keypoints to match frame dimensions
def scale_golden_keypoints(frame_width, frame_height):
    scaled_keypoints = golden_keypoints.copy()
    scaled_keypoints[:, 0] *= frame_width
    scaled_keypoints[:, 1] *= frame_height
    return scaled_keypoints

# Function to calculate Euclidean distance for pose accuracy
def calculate_pose_accuracy(user_keypoints, golden_keypoints):
    distances = np.linalg.norm(user_keypoints - golden_keypoints, axis=1)
    return distances.mean()

# Function to provide alignment feedback
def provide_alignment_feedback(user_keypoints, golden_keypoints, threshold=20):
    feedback = []
    keypoints = ['left_shoulder', 'right_shoulder', 'left_hip', 'right_hip',
                 'left_knee', 'right_knee', 'left_ankle', 'right_ankle']
    
    for i, (user_kp, golden_kp) in enumerate(zip(user_keypoints, golden_keypoints)):
        diff_x, diff_y = user_kp - golden_kp
        direction = []
        if abs(diff_x) > threshold:
            direction.append("move left" if diff_x > 0 else "move right")
        if abs(diff_y) > threshold:
            direction.append("move up" if diff_y > 0 else "move down")
        if direction:
            feedback.append(f"{keypoints[i]}: {' and '.join(direction)}")
    
    return feedback if feedback else ["Great Job!"]

# Load the reference video
reference_video_path = 'Reference-Video1.mp4'  # Update with actual video path
reference_cap = cv2.VideoCapture(reference_video_path)

# Get native resolution for the reference video
ref_width = int(reference_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
ref_height = int(reference_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Initialize camera feed
user_cap = cv2.VideoCapture(0)

if not user_cap.isOpened():
    print("Error: User camera feed could not be opened.")
if not reference_cap.isOpened():
    print("Error: Reference video could not be loaded.")

alignment_threshold = 20  # Pose accuracy threshold
total_accuracy = 0
frame_count = 0
pause_ref_video = False
last_ref_frame = None

while reference_cap.isOpened() and user_cap.isOpened():
    # Capture the reference video frame only if pose is correct
    if not pause_ref_video:
        ref_ret, ref_frame = reference_cap.read()
        if not ref_ret:
            print("Reference video completed.")
            break  # End video playback after one run
        last_ref_frame = ref_frame.copy()  # Save last frame for pausing
    else:
        # Keep displaying the last frame if reference video is paused
        ref_frame = last_ref_frame

    # Capture the user camera frame
    user_ret, user_frame = user_cap.read()
    if not user_ret:
        print("Error reading from user camera. Exiting.")
        break
    
    # Flip the user frame for a selfie view
    user_frame = cv2.flip(user_frame, 1)
    h, w, _ = user_frame.shape  # Full-sized user frame

    # Convert user frame to RGB and process for pose landmarks
    rgb_frame = cv2.cvtColor(user_frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)
    
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(user_frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        # Extract keypoints for yoga alignment
        user_keypoints = np.array([
            [results.pose_landmarks.landmark[i].x * w, results.pose_landmarks.landmark[i].y * h]
            for i in [11, 12, 23, 24, 25, 26, 27, 28]  # Indices for shoulders, hips, knees, ankles
        ])
        
        # Calculate pose accuracy and provide feedback
        accuracy = calculate_pose_accuracy(user_keypoints, scale_golden_keypoints(w, h))
        total_accuracy += accuracy
        frame_count += 1
        feedback_text = provide_alignment_feedback(user_keypoints, scale_golden_keypoints(w, h))
        
        # Control video play/pause based on pose accuracy
        if accuracy < alignment_threshold:
            feedback_display_text = "Great Job!"
            color = (0, 255, 0)
            pause_ref_video = False  # Continue advancing the reference video
        else:
            color = (0, 0, 255)
            pause_ref_video = True  # Pause the reference video

        # Display feedback text on user frame, line-by-line
        y_offset = 50
        for line in feedback_text:
            cv2.putText(user_frame, line, (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 1)
            y_offset += 25  # Space out each feedback line

        # Display pose accuracy
        cv2.putText(user_frame, f"Pose Accuracy: {accuracy:.2f}", (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

    # Resize reference video if necessary to fit in the display area
    if ref_frame.shape[0] > h // 2:
        ref_frame = cv2.resize(ref_frame, (w // 2, h // 2))  # Adjust to fit if too large

    # Overlay reference video in the top-right corner of the user frame
    user_frame[10:10 + ref_frame.shape[0], w - 10 - ref_frame.shape[1]:w - 10] = ref_frame

    # Display the user frame in a fullscreen window optimized for portrait mode
    cv2.namedWindow("Yoga Alignment Assistant", cv2.WINDOW_NORMAL)
    cv2.setWindowProperty("Yoga Alignment Assistant", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    cv2.imshow("Yoga Alignment Assistant", user_frame)
    
    # Quit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Calculate the final score
if frame_count > 0:
    average_accuracy = total_accuracy / frame_count
    score = max(0, 100 - average_accuracy)  # Scaled score out of 100
    print(f"Exercise Score: {score:.2f}%")
else:
    print("No frames were processed.")

# Release resources
user_cap.release()
reference_cap.release()
cv2.destroyAllWindows()
pose.close()


Exercise Score: 0.00%


In [1]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose Estimation
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

# Define key landmarks for yoga alignment (e.g., Warrior II)
golden_keypoints = np.array([
    [0.45, 0.3],  # left_shoulder
    [0.55, 0.3],  # right_shoulder
    [0.4, 0.5],   # left_hip
    [0.6, 0.5],   # right_hip
    [0.35, 0.7],  # left_knee
    [0.65, 0.7],  # right_knee
    [0.3, 0.9],   # left_ankle
    [0.7, 0.9],   # right_ankle
])

# Scale golden keypoints to match frame dimensions
def scale_golden_keypoints(frame_width, frame_height):
    scaled_keypoints = golden_keypoints.copy()
    scaled_keypoints[:, 0] *= frame_width
    scaled_keypoints[:, 1] *= frame_height
    return scaled_keypoints

# Function to calculate Euclidean distance for pose accuracy
def calculate_pose_accuracy(user_keypoints, golden_keypoints):
    distances = np.linalg.norm(user_keypoints - golden_keypoints, axis=1)
    return distances.mean()

# Function to provide alignment feedback
def provide_alignment_feedback(user_keypoints, golden_keypoints, threshold=20):
    feedback = []
    keypoints = ['left_shoulder', 'right_shoulder', 'left_hip', 'right_hip',
                 'left_knee', 'right_knee', 'left_ankle', 'right_ankle']
    
    for i, (user_kp, golden_kp) in enumerate(zip(user_keypoints, golden_keypoints)):
        diff_x, diff_y = user_kp - golden_kp
        direction = []
        if abs(diff_x) > threshold:
            direction.append("move left" if diff_x > 0 else "move right")
        if abs(diff_y) > threshold:
            direction.append("move up" if diff_y > 0 else "move down")
        if direction:
            feedback.append(f"{keypoints[i]}: {' and '.join(direction)}")
    
    return feedback if feedback else ["Great Job!"]

# Load the reference video
reference_video_path = 'WhatsApp Video 2024-10-26 at 19.38.12_ccc7f76d.mp4'  # Update with actual video path
reference_cap = cv2.VideoCapture(reference_video_path)

# Get native resolution for the reference video
ref_width = int(reference_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
ref_height = int(reference_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Initialize camera feed
user_cap = cv2.VideoCapture(0)

if not user_cap.isOpened():
    print("Error: User camera feed could not be opened.")
if not reference_cap.isOpened():
    print("Error: Reference video could not be loaded.")

alignment_threshold = 84  # Pose accuracy threshold
total_accuracy = 0
frame_count = 0
pause_ref_video = False
last_ref_frame = None

while reference_cap.isOpened() and user_cap.isOpened():
    # Capture the reference video frame only if pose is correct
    if not pause_ref_video:
        ref_ret, ref_frame = reference_cap.read()
        if not ref_ret:
            print("Reference video completed.")
            break  # End video playback after one run
        last_ref_frame = ref_frame.copy()  # Save last frame for pausing
    else:
        # Keep displaying the last frame if reference video is paused
        ref_frame = last_ref_frame

    # Capture the user camera frame
    user_ret, user_frame = user_cap.read()
    if not user_ret:
        print("Error reading from user camera. Exiting.")
        break
    
    # Flip the user frame for a selfie view
    user_frame = cv2.flip(user_frame, 1)
    h, w, _ = user_frame.shape  # Full-sized user frame

    # Convert user frame to RGB and process for pose landmarks
    rgb_frame = cv2.cvtColor(user_frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)
    
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(user_frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        # Extract keypoints for yoga alignment
        user_keypoints = np.array([
            [results.pose_landmarks.landmark[i].x * w, results.pose_landmarks.landmark[i].y * h]
            for i in [11, 12, 23, 24, 25, 26, 27, 28]  # Indices for shoulders, hips, knees, ankles
        ])
        
        # Calculate pose accuracy and provide feedback
        accuracy = calculate_pose_accuracy(user_keypoints, scale_golden_keypoints(w, h))
        total_accuracy += accuracy
        frame_count += 1
        feedback_text = provide_alignment_feedback(user_keypoints, scale_golden_keypoints(w, h))
        
        # Control video play/pause based on pose accuracy
        if accuracy < alignment_threshold:
            feedback_display_text = "Great Job!"
            color = (0, 255, 0)
            pause_ref_video = False  # Continue advancing the reference video
        else:
            color = (0, 0, 255)
            pause_ref_video = True  # Pause the reference video

        # Display feedback text on user frame, line-by-line with smaller font
        

        y_offset = 50
        for line in feedback_text:
            cv2.putText(user_frame, line, (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
            y_offset += 18  # Space out each feedback line

        # Display pose accuracy with smaller font
        cv2.putText(user_frame, f"Pose Accuracy: {accuracy:.2f}", (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

    # Resize reference video if necessary to fit in the display area
    if ref_frame.shape[0] > h // 2:
        ref_frame = cv2.resize(ref_frame, (w // 2, h // 2))  # Adjust to fit if too large

    # Overlay reference video in the top-right corner of the user frame
    user_frame[10:10 + ref_frame.shape[0], w - 10 - ref_frame.shape[1]:w - 10] = ref_frame

    # Display the user frame in a fullscreen window optimized for portrait mode
    cv2.namedWindow("Yoga Alignment Assistant", cv2.WINDOW_NORMAL)
    cv2.setWindowProperty("Yoga Alignment Assistant", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    cv2.imshow("Yoga Alignment Assistant", user_frame)
    
    # Quit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Calculate the final score
if frame_count > 0:
    average_accuracy = total_accuracy / frame_count
    score = max(0, 100 - average_accuracy)  # Scaled score out of 100
    print(f"Exercise Score: {score:.2f}%")
else:
    print("No frames were processed.")

# Release resources
user_cap.release()
reference_cap.release()
cv2.destroyAllWindows()
pose.close()


c:\Users\Arjun\miniconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Exercise Score: 0.00%
